# 朴素贝叶斯

## 使用朴素贝叶斯进行文档分类

## 案例1 文本分类

    以社区在线留言板为背景，实现判断言论是否含有侮辱性文字。


> 1 从文中构建词向量

In [31]:
def loadDataSet():
    """
    postingList:实验样本
    classVec:
        label=1 有侮辱性文字
        label=0 正常言论
    """
    postingList = [
        ['这', '个','菜', '好', '吃'],
        ['你', '他', '妈', '的', '有', '病', '吧'],
        ['真', '的', '好', '吃', '再', '来', ],
        ['最', '难', '吃', '的', '饭', '了', ],
        
#         ['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
#         ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
#         ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
#         ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
#         ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
#         ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']
    ]
    # 'my dog has flea problems help please'.split()
    # 'maybe not take him to dog park stupid'.split()
    # 'my dalmation is so cute I love him'.split()
    # 'stop posting stupid worthless garbage'.split()
    # 'mr licks ate my steak how to stop him'.split()
    # 'quit buying worthless dog food stupid'.split()
    classVec = [0, 1, 0, 1]
    return postingList, classVec
    
    
def createVocabList(dataSet):
    """
    获取dataSet中的所有单词集合
    """
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)


def setOfWords2Vec(vocabList, inputSet):
    """
    将文档转成文档向量
    """
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print('the word: {} is not in my Vocabulary!'.format(word))
    return returnVec


def setOfWords2Vec(vocabList, inputSet):
    """
    创建词袋模型
    """
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
        else:
            print('the word: {} is not in my Vocabulary!'.format(word))
    return returnVec

> 从词向量计算概率

In [32]:
import numpy as np

def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    p0Num, p1Num = np.ones(numWords), np.ones(numWords)
    p0Denom, p1Denom = 2.0, 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = np.log(p1Num / p1Denom)
    p0Vect = np.log(p0Num / p0Denom)
    return p0Vect, p1Vect, pAbusive


> 朴素贝叶斯分类函数

In [33]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    
    if p1 > p0:
        return 1
    else:
        return 0

> main

In [36]:
def testingNB():
    # get train data
    listOPosts, listClasses = loadDataSet()
    # get word vector
    myVocabList = createVocabList(listOPosts)
    # word to vector
    trainMat = []
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    
    # 
    p0V, p1V, pAb = trainNB0(np.array(trainMat), np.array(listClasses))
    
#     testEntry = 'love my dalmation'.split()
    testEntry = ['有', '病', '吧']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print('classified as:', classifyNB(thisDoc, p0V, p1V, pAb))
    
#     testEntry = 'stupid garbage'.split()
    testEntry = ['好', '吃', '的']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print('classified as:', classifyNB(thisDoc, p0V, p1V, pAb))

In [37]:
testingNB()

classified as: 0
classified as: 1


## 案例2 使用NB过滤垃圾邮件

1. 加载25个正常邮件和垃圾邮件
2. 将每个邮件转为词向量
3. train

In [138]:
import os
import re

from sklearn.model_selection import train_test_split

root_path = r'data/email'

def textParse(text):
    """
    对邮件进行分割，保留长度大于2的单词。
    """
    pattern = re.compile(r'\W*')
    return [doc.lower() for doc in pattern.split(text) if len(doc) > 2]


def getWordVector(docList):
    """
    get word vector
    """
    word_vector = set()
    for doc in docList:
        word_vector = word_vector | set(doc)
    return list(word_vector)


def wordEncord(word_vector, docList):
    """
    按照生成的词向量，对样本进行编码
    """
    trainMat = np.zeros((len(docList), len(word_vector)))
    for index, doc in enumerate(docList):
        trainMat[index][[word_vector.index(i) for i in doc]] = 1
    return trainMat


def myNB(train_datas, labels):
    """
    对各类别出现的单词进行求和，然后求出每个单词在该类别中的概率
    """
    train_datas = np.asarray(train_datas)
    # 初始化为1，而不是零是为了防止，在进行概率相乘时出现0值
    vec0, vec1 = np.ones(len(train_datas[0])), np.ones(len(train_datas[0]))
    sum0, sum1 = 2., 2.
    for index, label in enumerate(labels):
        if label == 0:
            vec0 = vec0 + train_datas[index]
            sum0 += sum(train_datas[index])
        else:
            vec1 = vec1 + train_datas[index]
            sum1 += sum(train_datas[index])
    c1 = sum(labels) * 1. / len(labels)
    return np.log(vec0 / sum0), np.log(vec1 / sum1), c1


def myClassifyNB(docVector, v0, v1, c1):
    p1 = np.sum(np.array(docVector) * v1) + np.log(c1)
    p0 = np.sum(np.array(docVector) * v0) + np.log(1 - c1)
    print('p1:%.2f, \tp0:%.2f' % (p1, p0))
    if p1 > p0:
        return 1
    else:
        return 0
    
    
def spamTest():
    docList = []
    classList = []
    for i in range(1, 26):
        with open(os.path.join(root_path, 'spam/%d.txt' % i), encoding='ISO-8859-1') as f:
            docList.append(textParse(f.read()))
            classList.append(1)
        with open(os.path.join(root_path, 'ham/%d.txt' % i), encoding='ISO-8859-1') as f:
            docList.append(textParse(f.read()))
            classList.append(0)
            
    word_vector = getWordVector(docList)
    trainMat = wordEncord(word_vector, docList)
    
    # 对数据集进行划分
    xtrian, xtest, ytrain, ytest = train_test_split(trainMat, classList, test_size=0.2, random_state=10)
    
    # 构建贝叶斯模型
    v0, v1, c1 = myNB(xtrian, ytrain)
    num_err = 0
    for x, y in zip(xtest, ytest):
        pred_y = myClassifyNB(x, v0, v1, c1)
        if pred_y != y:
            num_err += 1
    print('accu:%.2f' % (1 - num_err / len(ytest)))

In [139]:
spamTest()

p1:-87.97, 	p0:-82.64
p1:-136.12, 	p0:-130.36
p1:-136.74, 	p0:-186.53
p1:-214.10, 	p0:-260.91
p1:-23.15, 	p0:-19.82
p1:-174.84, 	p0:-227.01
p1:-165.72, 	p0:-169.28
p1:-136.74, 	p0:-186.53
p1:-142.29, 	p0:-139.59
p1:-121.04, 	p0:-148.25
accu:0.90


D:\softfiles\programs\anaconda3\lib\site-packages\ipykernel\__main__.py:13: FutureWarning: split() requires a non-empty pattern match.


## 案例3 从个人广告中获取区域倾向

In [140]:
import feedparser